## Ejercicio 9

Se ha realizado un análisis químico a tres tipos distintos de vinos producidos en una misma región de Italia. El número de muestras considerado es el siguiente:

$$
	\text{Tipo 1} \rightarrow 59~\text{muestras}, \quad\quad \text{Tipo 2} \rightarrow 71~\text{muestras}, \quad\quad \text{Tipo 3} \rightarrow 48~\text{muestras}
$$


El archivo **vinos.csv** permite observar los resultados de este análisis. Cada fila representa una muestra distinta y está formada, en primer lugar, por el número del tipo al cual pertenece el vino analizado, seguido por los 13 atributos que lo caracterizan.  

Por ejemplo, la siguiente fila:

$$
	2, ~12.29, ~3.17, ~2.21, ~18, ~88, ~2.85, ~2.99, ~0.45, ~2.81, ~2.3, ~1.42, ~2.83, ~406
$$


es el resultado del análisis de un vino correspondiente al **tipo 2** (primer valor de la fila), seguido por 13 valores separados por comas que indican los niveles de las mediciones realizadas a dicho vino.

In [2]:
import pandas as pd

DATOS_DIR   = '../../Datos/'
data = pd.read_csv(DATOS_DIR + 'vinos.csv', sep=';')
data

,Class,Alcohol,Malic acid,Ash,Alcalinity of ash,Magnesium,Total phenols,Flavanoids,Nonflavanoid phenols,Proanthocyanins,Color intensity,Hue,OD280/OD315,Proline
0,1,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,1,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,1,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,1,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,3,13.71,5.65,2.45,20.5,95,1.68,0.61,0.52,1.06,7.70,0.64,1.74,740
174,3,13.40,3.91,2.48,23.0,102,1.80,0.75,0.43,1.41,7.30,0.70,1.56,750
175,3,13.27,4.28,2.26,20.0,120,1.59,0.69,0.43,1.35,10.20,0.59,1.56,835
176,3,13.17,2.59,2.37,20.0,120,1.65,0.68,0.53,1.46,9.30,0.60,1.62,840


### a) 

Entrene una red neuronal formada por una única neurona para clasificar los vinos de **Tipo 1**.

Realice **30 ejecuciones independientes** utilizando el **50%, 60%, 70%, 80% y 90%** de los ejemplos como entrenamiento y el resto como testeo. Para cada porcentaje, indique la cantidad promedio de ejemplos correctamente clasificados en entrenamiento y en testeo. Calcule también el promedio y el desvío de la cantidad de iteraciones realizadas.

Utilice un máximo de 400 iteraciones y velocidades de aprendizaje 0.1, 0.2 y 0.3.


In [3]:
import numpy as np
import pandas as pd
from Fuentes.ClassNeuronaGral import *
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

MAX_ITER = 250
COTA = 1e-3

T = (data["Class"] == 1).astype(int)   
X = data.iloc[:, 1:]                  

scaler = MinMaxScaler()
X = scaler.fit_transform(X)

def run_experiments(X, T, FUN="sigmoid", COSTO="ECM", 
                    max_iter=1000, cota=1e-3,
                    train_sizes=[0.5, 0.6, 0.7, 0.8, 0.9],
                    alphas=[0.1, 0.2, 0.3],
                    n_runs=30):
    
    resultados = []
    
    for perc_for_training in train_sizes:
        for alpha in alphas:
            acc_train_runs, acc_test_runs, iters_runs = [], [], []

            for _ in range(n_runs):
                X_train, X_test, T_train, T_test = train_test_split(
                    X, T, train_size=perc_for_training, stratify=T, random_state=None
                )
                
                nnp = NeuronaGradiente(
                    alpha=alpha, n_iter=max_iter, cotaE=cota, FUN=FUN, COSTO=COSTO
                )
                nnp.fit(X_train, T_train.values)
                
                acc_train = nnp.accuracy(X_train, T_train.values)
                acc_test  = nnp.accuracy(X_test, T_test.values)
                
                acc_train_runs.append(acc_train)
                acc_test_runs.append(acc_test)
                iters_runs.append(nnp.iterations_)
            
            resultados.append({
                "train_size": perc_for_training,
                "alpha": alpha,
                "mean_acc_train": np.mean(acc_train_runs),
                "mean_acc_test": np.mean(acc_test_runs),
                "mean_iters": np.mean(iters_runs),
                "std_iters": np.std(iters_runs)
            })
        
        # Resumen general para cada train_size
        df_temp = pd.DataFrame([r for r in resultados if r["train_size"] == perc_for_training])
        resultados.append({
            "train_size": perc_for_training,
            "alpha": "general",
            "mean_acc_train": df_temp["mean_acc_train"].mean(),
            "mean_acc_test": df_temp["mean_acc_test"].mean(),
            "mean_iters": df_temp["mean_iters"].mean(),
            "std_iters": df_temp["std_iters"].mean()
        })
    
    return pd.DataFrame(resultados)

Analice los resultados obtenidos utilizando:

**i.** Función de activación ‘sigmoid’ y función de costo ‘ECM’ (error cuadrático medio)

In [4]:
run_experiments(X, T, FUN="sigmoid", COSTO="ECM", max_iter=MAX_ITER, cota=COTA)

,train_size,alpha,mean_acc_train,mean_acc_test,mean_iters,std_iters
0,0.5,0.1,0.991760,0.975655,250.0,0.0
1,0.5,0.2,0.997753,0.987266,250.0,0.0
2,0.5,0.3,0.999625,0.986891,250.0,0.0
3,0.5,general,0.996380,0.983271,250.0,0.0
4,0.6,0.1,0.994654,0.980093,250.0,0.0
5,0.6,0.2,0.998742,0.987037,250.0,0.0
6,0.6,0.3,1.000000,0.981481,250.0,0.0
7,0.6,general,0.997799,0.982870,250.0,0.0
8,0.7,0.1,0.994086,0.980864,250.0,0.0
9,0.7,0.2,0.999462,0.990123,250.0,0.0


**ii.** Función de activación ‘sigmoid’ y función de costo ‘EC_binaria’ (entropía cruzada binaria)

In [5]:
run_experiments(X, T, FUN="sigmoid", COSTO="EC_binaria", max_iter=MAX_ITER, cota=COTA)

,train_size,alpha,mean_acc_train,mean_acc_test,mean_iters,std_iters
0,0.5,0.1,0.992509,0.977528,250.0,0.0
1,0.5,0.2,1.000000,0.979026,250.0,0.0
2,0.5,0.3,0.999625,0.983895,250.0,0.0
3,0.5,general,0.997378,0.980150,250.0,0.0
4,0.6,0.1,0.993711,0.984259,250.0,0.0
5,0.6,0.2,0.998742,0.983333,250.0,0.0
6,0.6,0.3,1.000000,0.986111,250.0,0.0
7,0.6,general,0.997484,0.984568,250.0,0.0
8,0.7,0.1,0.995430,0.981481,250.0,0.0
9,0.7,0.2,1.000000,0.987654,250.0,0.0


**iii.** Función de activación ‘tanh’ y función de costo ‘ECM’ (error cuadrático medio)

In [6]:
T_tanh = 2*T - 1
run_experiments(X, T_tanh, FUN="tanh", COSTO="ECM", max_iter=MAX_ITER, cota=COTA)

,train_size,alpha,mean_acc_train,mean_acc_test,mean_iters,std_iters
0,0.5,0.1,1.0,0.988764,250.000000,0.000000
1,0.5,0.2,1.0,0.982772,232.300000,20.325928
2,0.5,0.3,1.0,0.984644,193.866667,19.845626
3,0.5,general,1.0,0.985393,225.388889,13.390518
4,0.6,0.1,1.0,0.987037,250.000000,0.000000
5,0.6,0.2,1.0,0.989352,243.366667,12.453870
6,0.6,0.3,1.0,0.987500,201.600000,21.240841
7,0.6,general,1.0,0.987963,231.655556,11.231571
8,0.7,0.1,1.0,0.987037,250.000000,0.000000
9,0.7,0.2,1.0,0.993210,249.233333,3.094978


> **Conclusión:** 